#DETECCION DE POSIBLE COVID DE COVID-19 A BASE DE LA VOZ

In [ ]:
#@title **MONTAR EL DRIVE** { display-mode: "form" }
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Cositas UIS/IA 1/Coughs')

Mounted at /content/drive


In [ ]:
#@title CARGA DE LIBRERIAS { display-mode: "form" }
import os
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split

2.7.0


In [ ]:
#@title Audio del dataset
file_name = 'N-sample (5).wav' #@param {type:"string"}
ipd.Audio('/content/drive/My Drive/Cositas UIS/IA 1/Coughs/'+file_name)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
def func_padding(file_name, pad_size=4000):
    if file_name.shape[0] > pad_size:
        return file_name[0: pad_size]
    else:
        return np.hstack((file_name, np.zeros(pad_size - file_name.shape[0])))

In [ ]:
def get_XyData(DATA_DIR = '/content/drive/My Drive/Cositas UIS/IA 1/Coughs/', pad_size=2000):

  X =[]
  y =[]
  
  for fname in os.listdir(DATA_DIR):
    struct = fname.split('-')
    if struct[0] == 'C':
      struct[0] = '1'
    else:
      struct[0] = '0'
    digit = struct[0]
    wav, sr = librosa.load(DATA_DIR + fname)
    padded = func_padding(wav, pad_size)
    X.append(padded)
    y.append(digit)   

  return np.vstack(X), np.array(y)

X, y = get_XyData()

print( 'X:', X.shape)
print( 'y:', y.shape)

X: (153, 2000)
y: (153,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
print(X_train.shape, X_test.shape)
print(X_train[0].shape)
print(y_train.shape, y_test.shape)

(91, 2000) (62, 2000)
(2000,)
(91,) (62,)


In [ ]:
y_train_ohe = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_test_ohe = tf.keras.utils.to_categorical(y_test, num_classes=2)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten( input_shape= X_train[0].shape),
    tf.keras.layers.Dense(128, activation='relu'),                             
    tf.keras.layers.Dense(2, activation='sigmoid')     #softmax for multiple classes               
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 2000)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               256128    
                                                                 
 dense_3 (Dense)             (None, 2)                 258       
                                                                 
Total params: 256,386
Trainable params: 256,386
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(), 
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train_ohe, epochs=5)

Epoch 1/5
3/3 [==============================] - 1s 8ms/step - loss: 0.6931 - accuracy: 0.5714
Epoch 2/5
3/3 [==============================] - 0s 6ms/step - loss: 0.6923 - accuracy: 0.6813
Epoch 3/5
3/3 [==============================] - 0s 6ms/step - loss: 0.6915 - accuracy: 0.6813
Epoch 4/5
3/3 [==============================] - 0s 6ms/step - loss: 0.6907 - accuracy: 0.6813
Epoch 5/5
3/3 [==============================] - 0s 6ms/step - loss: 0.6899 - accuracy: 0.6813


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test_ohe)

2/2 [==============================] - 0s 7ms/step - loss: 0.6870 - accuracy: 0.7903
